In [500]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats

In [501]:
SP = pd.read_csv('data/SnP500.csv').dropna()
SP.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1,1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
2,1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
3,1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
4,1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0


In [502]:
# SP.rename(columns={'GOLDAMGBD228NLBM': 'Close'}, inplace=True)
# SP.Close = SP.Close.apply(pd.to_numeric, errors='coerce')
# SP.Close = SP.Close.fillna(method='ffill')

In [503]:
SP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23283 entries, 0 to 23282
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       23283 non-null  object 
 1   Open       23283 non-null  float64
 2   High       23283 non-null  float64
 3   Low        23283 non-null  float64
 4   Close      23283 non-null  float64
 5   Adj Close  23283 non-null  float64
 6   Volume     23283 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.4+ MB


In [504]:
i = 1
f = [1, 2]

for i in range(15):
    i = f[-1] + f[-2]
    f.append(i)
    
print(f)

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584]


In [505]:
df = []

for i in range(15000):
    chnd = (SP.Close.iloc[-2 - i - 5]/SP.Close.iloc[-1 - i] - 1)*100
    row = []
    if chnd >= 0:
        row.append(1)
    else:
        row.append(0)
    for j in (2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610):
        row.append(SP.Close.iloc[-j - i - 5])
    df.append(row)
    

In [506]:
df = pd.DataFrame(df)
df.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,3500.310059,3508.010010,3478.729980,3397.159912,3372.850098,3306.510010,3215.570068,3124.739990,2863.389893,3327.709961,2940.250000,2748.929932,2614.449951
1,1,3508.010010,3484.550049,3443.620117,3385.510010,3373.429932,3294.610107,3226.560059,3066.590088,2878.479980,3345.780029,2976.739990,2771.449951,2581.879883
2,1,3484.550049,3478.729980,3431.280029,3374.850098,3380.350098,3271.120117,3197.520020,3041.310059,2836.739990,3334.689941,2961.790039,2789.649902,2640.870117
3,1,3478.729980,3443.620117,3397.159912,3389.780029,3333.689941,3246.219971,3155.219971,3002.100098,2797.800049,3297.590088,2977.620117,2792.810059,2605.000000
4,0,3443.620117,3431.280029,3385.510010,3381.989990,3360.469971,3258.439941,3185.040039,3190.139893,2799.310059,3248.919922,2984.870117,2803.689941,2612.620117


In [507]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       15000 non-null  int64  
 1   1       15000 non-null  float64
 2   2       15000 non-null  float64
 3   3       15000 non-null  float64
 4   4       15000 non-null  float64
 5   5       15000 non-null  float64
 6   6       15000 non-null  float64
 7   7       15000 non-null  float64
 8   8       15000 non-null  float64
 9   9       15000 non-null  float64
 10  10      15000 non-null  float64
 11  11      15000 non-null  float64
 12  12      15000 non-null  float64
 13  13      15000 non-null  float64
dtypes: float64(13), int64(1)
memory usage: 1.6 MB


In [508]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, classification_report, precision_recall_curve, confusion_matrix

In [509]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['0'], 1), df['0'], random_state=0)

In [510]:
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    

continuous_columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']

final_transformers = list()
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

feats = FeatureUnion(final_transformers)
feature_processing = Pipeline([('feats', feats)])

pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)
preds = pipeline.predict_proba(X_test)[:, 1]

In [518]:
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)
# preds = rf.predict_proba(X_test)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision/50 * recall) / (precision/50 + recall)
# fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%.3f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

roc_auc = roc_auc_score(y_true=y_test, y_score=preds)
print(roc_auc)

Best Threshold=0.539, F-Score=0.027, Precision=0.723, Recall=0.165
0.6530233177394428


In [512]:
preds

array([0.40719492, 0.45421928, 0.41803849, ..., 0.54959133, 0.39553911,
       0.48419821])